In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Input, Dropout

In [5]:
data = pd.read_csv('ABNB_stock_data.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst = True)
data.set_index('Date', inplace=True)
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-12-10,146.000000,165.000000,141.250000,144.710007,144.710007,70447500
2020-12-11,146.550003,151.500000,135.100006,139.250000,139.250000,26980800
2020-12-14,135.000000,135.300003,125.160004,130.000000,130.000000,16966100
2020-12-15,126.690002,127.599998,121.500000,124.800003,124.800003,10914400
2020-12-16,125.830002,142.000000,124.910004,137.990005,137.990005,20409600


In [6]:
df1 = pd.DataFrame(data['Close'])
data = df1.values
train_test_split = int(np.ceil(0.65 * len(data)))
train_data = data[:train_test_split]
test_data = data[train_test_split:]
train_data.shape, test_data.shape

scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

In [7]:
x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))

In [8]:
model = Sequential()
model.add(GRU(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(GRU(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=200, batch_size=5)

D:\anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 0.4880
Epoch 2/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.1416
Epoch 3/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.1045
Epoch 4/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0821
Epoch 5/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0958
Epoch 6/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - loss: 0.0703
Epoch 7/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0840
Epoch 8/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0583
Epoch 9/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - loss: 0.0691
Epoch 10/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0516
Epoch 11/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0505
Epoch 12/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.0721
Epoch 13/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - loss: 0.0765
Epoch 14/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0651
Epoch 15/200
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - lo

In [9]:
y_train = scaler.inverse_transform(y_train)
y_train_predict = model.predict(x_train)
y_train_predict = scaler.inverse_transform(y_train_predict)

gru_r2 = r2_score(y_train, y_train_predict)
gru_mse = mean_squared_error(y_train, y_train_predict)
gru_mae = mean_absolute_error(y_train, y_train_predict)

print('MSE:', gru_mse)
print('MAE:', gru_mae)
print('R2 Score:', gru_r2)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step
MSE: 3.006558163166469
MAE: 1.2167232534670225
R2 Score: 0.9905218864278146
